# 實驗與測試區

### Import可能會用到的 套件/模組  (需執行)

In [1]:
import numpy as np
import torch
import torch.nn as nn

from munch import Munch

### Python index test

In [ ]:
### python index test
heatmap = np.array([[0.2, 0.5, 0.8], [0.9, 0.3, 0.6], [0.1, 0.7, 0.4]])
threshold = 0.5
print("heatmap:", heatmap)  # output: heatmap: [[0.2 0.5 0.8]
                            #                   [0.9 0.3 0.6]
                            #                   [0.1 0.7 0.4]]
heatmap[heatmap < threshold] = 0
print("heatmap:", heatmap)  # output: heatmap: [[0.2 0.5 0.8]
                            #                   [0.9 0.  0.6]
                            #                   [0.  0.7 0. ]]

### BCELoss test

In [ ]:
### BCELoss test
m = nn.Sigmoid()
# bce_loss = nn.BCELoss(reduction='sum')  # 沒有平均
bce_loss = nn.BCELoss()  # 有平均
output = torch.tensor([-1, -2, -3, 1, 2, 3], dtype=torch.float32)
output = m(output)  # 先處理一次 (將數值轉到0~1之間)
target = torch.tensor([0, 1, 0, 0, 0, 1], dtype=torch.float32)
loss = bce_loss(output, target)
print(loss)

### 模型並聯 (提供by ChatGPT)

In [ ]:
### 模型並聯
class ParallelConv(nn.Module):
    def __init__(self, in_channels, out_channels, num_convs):
        super(ParallelConv, self).__init__()
        self.num_convs = num_convs
        self.convs = nn.ModuleList([nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1) for i in range(num_convs)])

    def forward(self, x):
        outputs = []
        for i in range(self.num_convs):
            outputs.append(self.convs[i](x))
        return torch.cat(outputs, dim=1)

### Dictionary / Munch test

In [ ]:
### dictionary test
cfg = []

# test 1
cfg.append(
    dict(num_modules=1, num_branches=1, block='BOTTLENECK', num_blocks=(4), num_channels=(64, ))
)

# test 2
cfg.append(
    dict(
        MODEL = dict(
            NAME = "cls_hrnet",
            IMAGE_SIZE = (224, 224)
        )
    )
)

# test 3
cfg.append(
    Munch()
)
cfg[-1].MODEL = Munch()
cfg[-1].MODEL.NAME = "cls_hrnet"

# test 4
cfg.append(
    {
        "MODEL" : {
            "NAME" : "cls_hrnet",
            "IMAGE_SIZE" : (224, 224)
        }
    }
)


# Log Output
for i in range(len(cfg)):
    print("\n===")
    print(">> cfg[{}] : {}".format(i, cfg[i]))
    try:
        print(">> cfg[{}]['MODEL'] : {}".format(i, cfg[i]['MODEL']))
    except:
        print(">> cfg[{}]['MODEL'] : error".format(i))
    try:
        print(">> cfg[{}].MODEL : {}".format(i, cfg[i].MODEL))
    except:
        print(">> cfg[{}].MODEL : error".format(i))
    try:
        print(">> cfg[{}].MODEL.NAME : {}".format(i, cfg[i].MODEL.NAME))
    except:
        print(">> cfg[{}].MODEL.NAME : error".format(i))

### ONNX test (簡單模型視覺化)

In [6]:
### onnx test
BN_MOMENTUM = 0.1
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes, momentum=BN_MOMENTUM)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes, momentum=BN_MOMENTUM)
        self.conv3 = nn.Conv2d(planes, planes*self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes*self.expansion, momentum=BN_MOMENTUM)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)  # layer 1
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)  # layer 2
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)  # layer 3
        out = self.bn3(out)
        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)
        return out

def Convert_ONNX(model):
    # set the model to inference mode
    model.eval() 

    # Let's create a dummy input tensor
    dummy_input = torch.randn(1, 4, 5, 5)  

    # Export the model
    torch.onnx.export(model,  # model being run 
         dummy_input,  # model input (or a tuple for multiple inputs)
         "Bottleneck.onnx",  # where to save the model
         export_params=True,  # store the trained parameter weights inside the model file
         opset_version=10,  # the ONNX version to export the model to
         do_constant_folding=True,  # whether to execute constant folding for optimization 
         input_names = ['modelInput'],  # the model's input names
         output_names = ['modelOutput'],  # the model's output names
         dynamic_axes={'modelInput' : {0 : 'batch_size'}, 'modelOutput' : {0 : 'batch_size'}}  # variable length axes
    )
    print("\n>> Model has been converted to ONNX")

if __name__ == "__main__":
    model = Bottleneck(4, 1)
    Convert_ONNX(model)
    
    # from torch.utils.tensorboard import SummaryWriter
    # tb = SummaryWriter()
    # dummy_input = torch.randn(1, 4, 5, 5)
    # tb.add_graph(model, dummy_input)
    # tb.close()

### torch.zeros_like()

In [ ]:

heatmap = torch.randn(2, 5, 5)  # 隨便假設一個tensor
hint_heatmap = torch.zeros_like(heatmap)  # torch.zeros_like()裡面不只可以放"大小" 也可以放一個tesor然後自動取同樣大小
print(">> heatmap:\n{}".format(heatmap))
print(">> hint_heatmap:\n{}".format(hint_heatmap))

### def in def

In [ ]:
### def in def
def f1():
    print(">> i'm f1")
    
    def f2():
        print(">> i'm f2")
    
    return f2  # return a func
    # return f2()  # error

if __name__ == "__main__":
    func = f1()
    print("===")
    func()